In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4400.32
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 

**----------Load The Data---------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [26]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------VP Tree---------------------**

In [9]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**------VP Tree Node Size Optimization For Rebuild------**

**------------Run 1---------------------**

In [10]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 39.92it/s]


-------------------------------------------------------------
Build Time: 22.6983 sec, Search Time: 0.8518 sec
Leaf Size: 100, Total Time: 23.5500 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 53.25it/s]


-------------------------------------------------------------
Build Time: 20.7719 sec, Search Time: 0.6560 sec
Leaf Size: 200, Total Time: 21.4279 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 13.1580 sec, Search Time: 0.5787 sec
Leaf Size: 9,800, Total Time: 13.7367 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 60.41it/s]


-------------------------------------------------------------
Build Time: 12.9334 sec, Search Time: 0.5874 sec
Leaf Size: 9,900, Total Time: 13.5207 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 59.95it/s]

-------------------------------------------------------------
Build Time: 13.0491 sec, Search Time: 0.5825 sec
Leaf Size: 10,000, Total Time: 13.6316 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 22.6983 sec, Search Time: 0.8518 sec, Total Time: 23.5500 sec
Leaf Size: 200, Build Time: 20.7719 sec, Search Time: 0.6560 sec, Total Time: 21.4279 sec
Leaf Size: 300, Build Time: 19.6232 sec, Search Time: 0.5508 sec, Total Time: 20.1740 sec
Leaf Size: 400, Build Time: 19.5043 sec, Search Time: 0.5442 sec, Total Time: 20.0485 sec
Leaf Size: 500, Build Time: 18.3776 sec, Search Time: 0.5023 sec, Total Time: 18.8799 sec
Leaf Size: 600, Build Time: 18.3498 sec, Search Time: 0.4957 sec, Total Time: 18.8455 sec
Leaf Size: 700, Build Time: 18.0495 sec, Search Time: 0.4959 sec, Total Time: 18.5454 sec
Leaf Size: 800, Build Time: 18.3181 sec, Search Time: 0.5012 sec, Total Time: 18.8193 sec
Leaf Size: 900, Build Time: 18.2682 sec, Search Time: 0.4955 sec, Total Time: 18.76

**---------------------Run 2---------------**

In [13]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 42.33it/s]


-------------------------------------------------------------
Build Time: 22.4087 sec, Search Time: 0.7802 sec
Leaf Size: 100, Total Time: 23.1890 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 57.85it/s]


-------------------------------------------------------------
Build Time: 20.9910 sec, Search Time: 0.6222 sec
Leaf Size: 200, Total Time: 21.6132 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 13.0734 sec, Search Time: 0.5911 sec
Leaf Size: 9,800, Total Time: 13.6645 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 57.49it/s]


-------------------------------------------------------------
Build Time: 13.0028 sec, Search Time: 0.5951 sec
Leaf Size: 9,900, Total Time: 13.5980 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 59.32it/s]

-------------------------------------------------------------
Build Time: 13.1579 sec, Search Time: 0.5864 sec
Leaf Size: 10,000, Total Time: 13.7443 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 22.4087 sec, Search Time: 0.7802 sec, Total Time: 23.1890 sec
Leaf Size: 200, Build Time: 20.9910 sec, Search Time: 0.6222 sec, Total Time: 21.6132 sec
Leaf Size: 300, Build Time: 19.1913 sec, Search Time: 0.5242 sec, Total Time: 19.7156 sec
Leaf Size: 400, Build Time: 19.3649 sec, Search Time: 0.5271 sec, Total Time: 19.8920 sec
Leaf Size: 500, Build Time: 18.0676 sec, Search Time: 0.4914 sec, Total Time: 18.5590 sec
Leaf Size: 600, Build Time: 18.0220 sec, Search Time: 0.4788 sec, Total Time: 18.5008 sec
Leaf Size: 700, Build Time: 18.1021 sec, Search Time: 0.4813 sec, Total Time: 18.5834 sec
Leaf Size: 800, Build Time: 17.9979 sec, Search Time: 0.4852 sec, Total Time: 18.4831 sec
Leaf Size: 900, Build Time: 18.0105 sec, Search Time: 0.4791 sec, Total Time: 18.48

**-----Opt Node Size-----**

In [14]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000]
  )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [23.55  ,21.4279,20.174 ,20.0485,18.8799,18.8455,18.5454,18.8193,18.7637,
 17.4846,17.4536,17.561 ,17.8025,17.3972,17.5158,17.6983,17.6762,17.4827,
 17.6549,16.3957,16.2168,16.6532,16.4529,16.343 ,16.5777,16.4022,16.3588,
 16.5613,16.2124,16.458 ,16.0175,16.3141,16.2464,16.3973,16.3919,16.3657,
 16.3038,16.5188,16.4255,14.9894,14.9872,14.8654,14.816 ,14.7325,14.8808,
 14.9743,14.9555,14.8661,15.0497,15.2407,14.9913,14.9686,15.1618,15.2348,
 15.0201,15.0952,15.1261,14.9264,14.9868,15.0365,14.9574,14.842 ,15.0478,
 15.0767,14.9515,14.9254,14.9895,15.1435,15.0529,14.8922,14.9544,14.8698,
 14.9395,15.0071,14.9924,14.9363,14.9826,14.8113,13.6752,13.6683,13.6584,
 13.6139,13.5721,13.6885,13.754 ,13.5893,13.637 ,13.5198,13.517 ,13.5771,
 13.6501,13.5562,13.5955,13.5538,13.6073,13.6148,13.4774,13.7367,13.5207,
 13.6316]
    )

total_times_2 = np.array(
    [23.189 ,21.6132,19.7156,19.892 ,18.559 ,18.5008,18.5834,18.4831,18.4896,
 17.3071,17.3355,17.2819,17.4387,17.3665,17.2809,17.2322,17.4401,17.47  ,
 17.3156,16.8142,16.2971,16.0904,16.4768,16.3114,16.1332,16.3359,16.5907,
 16.914 ,18.0103,19.7498,20.7702,21.1792,19.7549,20.7395,20.2322,19.1281,
 18.5984,18.749 ,17.8109,16.5086,17.8499,18.1376,17.9566,15.9734,15.1628,
 15.0161,15.0438,15.1987,15.3149,15.1729,15.4683,15.4787,15.6261,15.5928,
 15.5627,15.8261,15.511 ,15.5676,15.7482,15.4897,15.3177,15.528 ,15.4059,
 15.3356,15.1744,15.2376,15.2503,15.4633,15.4899,15.3846,15.4318,15.3372,
 15.305 ,15.2147,15.2403,15.1483,15.1079,15.2557,14.0952,13.9856,14.0366,
 14.0479,13.9334,13.807 ,13.6738,13.7856,13.6189,13.4827,13.6976,13.5479,
 13.5989,13.6382,13.4833,13.603 ,13.739 ,13.7779,13.6709,13.6645,13.598 ,
 13.7443]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 13.5012
Minimum found at leaf size: 8800


**-----------Epsilon Optimization For Rebuild---------**

**---------------Run 1---------------------**

In [15]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 8800
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 58.81it/s]


-------------------------------------------------------------
Build Time: 13.4086 sec, Search Time: 0.5897 sec
Epsilon: 0.0, Total Time: 13.9983 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 281.07it/s]


-------------------------------------------------------------
Build Time: 4.0560 sec, Search Time: 0.0975 sec
Epsilon: 0.5, Total Time: 4.1535 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.1035 sec, Search Time: 0.0036 sec
Epsilon: 24.0, Total Time: 2.1072 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 55406.92it/s]


-------------------------------------------------------------
Build Time: 2.0631 sec, Search Time: 0.0024 sec
Epsilon: 24.5, Total Time: 2.0655 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 36361.54it/s]

-------------------------------------------------------------
Build Time: 2.0824 sec, Search Time: 0.0024 sec
Epsilon: 25.0, Total Time: 2.0848 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.4086 sec, Search Time: 0.5897 sec, Total Time: 13.9983 sec
Epsilon: 0.5, Build Time: 4.0560 sec, Search Time: 0.0975 sec, Total Time: 4.1535 sec
Epsilon: 1.0, Build Time: 2.6006 sec, Search Time: 0.0363 sec, Total Time: 2.6369 sec
Epsilon: 1.5, Build Time: 2.2066 sec, Search Time: 0.0080 sec, Total Time: 2.2145 sec
Epsilon: 2.0, Build Time: 2.1551 sec, Search Time: 0.0077 sec, Total Time: 2.1629 sec
Epsilon: 2.5, Build Time: 2.0623 sec, Search Time: 0.0056 sec, Total Time: 2.0679 sec
Epsilon: 3.0, Build Time: 2.1239 sec, Search Time: 0.0042 sec, Total Time: 2.1281 sec
Epsilon: 3.5, Build Time: 2.1135 sec, Search Time: 0.0036 sec, Total Time: 2.1171 sec
Epsilon: 4.0, Build Time: 2.0404 sec, Search Time: 0.0029 sec, Total Time: 2.0433 sec
Epsilon: 4.5, Build Time: 2.10

**-------------Run 2------------------------**

In [17]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 8800
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 68.67it/s]


-------------------------------------------------------------
Build Time: 13.2287 sec, Search Time: 0.5444 sec
Epsilon: 0.0, Total Time: 13.7731 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 291.62it/s]


-------------------------------------------------------------
Build Time: 4.1058 sec, Search Time: 0.0939 sec
Epsilon: 0.5, Total Time: 4.1997 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.0429 sec, Search Time: 0.0027 sec
Epsilon: 24.0, Total Time: 2.0457 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 34211.29it/s]


-------------------------------------------------------------
Build Time: 2.0958 sec, Search Time: 0.0024 sec
Epsilon: 24.5, Total Time: 2.0982 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 27934.09it/s]

-------------------------------------------------------------
Build Time: 2.0566 sec, Search Time: 0.0029 sec
Epsilon: 25.0, Total Time: 2.0595 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.2287 sec, Search Time: 0.5444 sec, Total Time: 13.7731 sec
Epsilon: 0.5, Build Time: 4.1058 sec, Search Time: 0.0939 sec, Total Time: 4.1997 sec
Epsilon: 1.0, Build Time: 2.6168 sec, Search Time: 0.0393 sec, Total Time: 2.6561 sec
Epsilon: 1.5, Build Time: 2.2426 sec, Search Time: 0.0078 sec, Total Time: 2.2504 sec
Epsilon: 2.0, Build Time: 2.1328 sec, Search Time: 0.0076 sec, Total Time: 2.1404 sec
Epsilon: 2.5, Build Time: 2.0240 sec, Search Time: 0.0059 sec, Total Time: 2.0299 sec
Epsilon: 3.0, Build Time: 2.0851 sec, Search Time: 0.0042 sec, Total Time: 2.0893 sec
Epsilon: 3.5, Build Time: 2.0650 sec, Search Time: 0.0039 sec, Total Time: 2.0688 sec
Epsilon: 4.0, Build Time: 2.0729 sec, Search Time: 0.0035 sec, Total Time: 2.0765 sec
Epsilon: 4.5, Build Time: 2.08

**-----------Epsilon Optimization For Rebuild------------**

In [18]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [13.9983, 4.1535, 2.6369, 2.2145, 2.1629, 2.0679, 2.1281, 2.1171, 2.0433,
  2.1065, 2.0914, 2.0649, 2.1316, 2.0924, 2.0793, 2.1337, 2.0653, 2.0874,
  2.0904, 2.1152, 2.0904, 2.1261, 2.0843, 2.0774, 2.121 , 2.1166, 2.0925,
  2.0691, 2.0775, 2.0852, 2.1022, 2.0624, 2.0815, 2.0556, 2.0791, 2.099 ,
  2.087 , 2.088 , 2.0709, 2.0697, 2.088 , 2.0875, 2.0898, 2.05  , 2.0778,
  2.0673, 2.0907, 2.0817, 2.1072, 2.0655, 2.0848]
    )

total_times_2 = np.array(
    [13.7731, 4.1997, 2.6561, 2.2504, 2.1404, 2.0299, 2.0893, 2.0688, 2.0765,
  2.0906, 2.0782, 2.0838, 2.0802, 2.0454, 2.0843, 2.1403, 2.0694, 2.0997,
  2.0652, 2.0805, 2.0835, 2.0582, 2.1396, 2.0872, 2.0939, 2.0912, 2.114 ,
  2.0956, 2.0834, 2.0706, 2.0862, 2.0925, 2.0846, 2.0572, 2.1083, 2.0704,
  2.0728, 2.0813, 2.0708, 2.07  , 2.0693, 2.0785, 2.0497, 2.0594, 2.0846,
  2.0902, 2.055 , 2.0733, 2.0457, 2.0982, 2.0595]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 2.0489
Minimum found at epsilon size: 2.5


**-------------VP Node Optimization For Insert--------**

**--------------Run 1------------------**

In [20]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500
leaf_size_step = 100

leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.33it/s]


-------------------------------------------------------------
Build Time: 11.4384 sec, Search Time: 1.3654 sec
Leaf Size: 100, Total Time: 12.8037 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.24it/s]


-------------------------------------------------------------
Build Time: 11.2960 sec, Search Time: 1.3621 sec
Leaf Size: 200, Total Time: 12.6581 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.2938 sec, Search Time: 1.3463 sec
Leaf Size: 2,300, Total Time: 12.6401 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.20it/s]


-------------------------------------------------------------
Build Time: 11.5412 sec, Search Time: 1.3627 sec
Leaf Size: 2,400, Total Time: 12.9039 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.32it/s]

-------------------------------------------------------------
Build Time: 11.3128 sec, Search Time: 1.3603 sec
Leaf Size: 2,500, Total Time: 12.6731 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 11.4384 sec, Search Time: 1.3654 sec, Total Time: 12.8037 sec
Leaf Size: 200, Build Time: 11.2960 sec, Search Time: 1.3621 sec, Total Time: 12.6581 sec
Leaf Size: 300, Build Time: 11.3580 sec, Search Time: 1.3379 sec, Total Time: 12.6958 sec
Leaf Size: 400, Build Time: 11.3323 sec, Search Time: 1.3678 sec, Total Time: 12.7000 sec
Leaf Size: 500, Build Time: 11.3170 sec, Search Time: 1.3533 sec, Total Time: 12.6703 sec
Leaf Size: 600, Build Time: 11.1971 sec, Search Time: 1.3742 sec, Total Time: 12.5713 sec
Leaf Size: 700, Build Time: 11.2922 sec, Search Time: 1.3693 sec, Total Time: 12.6615 sec
Leaf Size: 800, Build Time: 11.4671 sec, Search Time: 1.3698 sec, Total Time: 12.8369 sec
Leaf Size: 900, Build Time: 11.3532 sec, Search Time: 1.3438 sec, Total Time: 12.696

**-------------Run 2---------------**

In [22]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 2500
leaf_size_step = 100

leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 16.26it/s]


-------------------------------------------------------------
Build Time: 11.3696 sec, Search Time: 1.4981 sec
Leaf Size: 100, Total Time: 12.8677 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 16.56it/s]


-------------------------------------------------------------
Build Time: 11.3184 sec, Search Time: 1.4755 sec
Leaf Size: 200, Total Time: 12.7939 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.1870 sec, Search Time: 1.3355 sec
Leaf Size: 2,300, Total Time: 12.5225 sec

24: === Testing with Leaf Size: 2,400, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 17.96it/s]


-------------------------------------------------------------
Build Time: 11.4484 sec, Search Time: 1.3831 sec
Leaf Size: 2,400, Total Time: 12.8316 sec

25: === Testing with Leaf Size: 2,500, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.46it/s]

-------------------------------------------------------------
Build Time: 11.2150 sec, Search Time: 1.3481 sec
Leaf Size: 2,500, Total Time: 12.5631 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 11.3696 sec, Search Time: 1.4981 sec, Total Time: 12.8677 sec
Leaf Size: 200, Build Time: 11.3184 sec, Search Time: 1.4755 sec, Total Time: 12.7939 sec
Leaf Size: 300, Build Time: 11.1854 sec, Search Time: 1.3408 sec, Total Time: 12.5262 sec
Leaf Size: 400, Build Time: 11.2201 sec, Search Time: 1.3580 sec, Total Time: 12.5781 sec
Leaf Size: 500, Build Time: 11.2872 sec, Search Time: 1.3572 sec, Total Time: 12.6444 sec
Leaf Size: 600, Build Time: 11.1605 sec, Search Time: 1.3698 sec, Total Time: 12.5303 sec
Leaf Size: 700, Build Time: 11.4806 sec, Search Time: 1.3416 sec, Total Time: 12.8222 sec
Leaf Size: 800, Build Time: 11.1343 sec, Search Time: 1.3695 sec, Total Time: 12.5038 sec
Leaf Size: 900, Build Time: 11.4030 sec, Search Time: 1.3438 sec, Total Time: 12.746

**------Optimum Node Size for Insertion----**

In [23]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
     [ 100, 200, 300, 400, 500, 600, 700, 800, 900,1000,1100,1200,1300,1400,
 1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
    )

# Total times from two runs (or versions)
total_times_1 = np.array(
    [12.8677,12.7939,12.5262,12.5781,12.6444,12.5303,12.8222,12.5038,12.7468,
 12.5098,12.4344,12.5957,12.602 ,12.5925,12.7857,12.6337,12.6187,12.7469,
 12.5822,12.8714,12.6189,12.6852,12.5225,12.8316,12.5631]
     )

total_times_2 = np.array(
    [12.8037,12.6581,12.6958,12.7   ,12.6703,12.5713,12.6615,12.8369,12.6969,
 12.7257,12.7453,12.58  ,12.5275,12.6132,12.5957,12.61  ,12.4088,12.5883,
 12.5441,12.6896,12.5644,12.5759,12.6401,12.9039,12.6731]
    )

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 12.5137
Minimum found at leaf size: 1700


**------Epsilon Optimization For Insertion------------**

**----------Run 1-----------------**

In [25]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 19.30it/s]


-------------------------------------------------------------
Build Time: 11.3433 sec, Search Time: 1.3332 sec
Epsilon: 0.0, Total Time: 12.6765 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 21.40it/s]


-------------------------------------------------------------
Build Time: 11.3993 sec, Search Time: 1.2424 sec
Epsilon: 0.5, Total Time: 12.6417 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.4579 sec, Search Time: 1.3692 sec
Epsilon: 24.0, Total Time: 12.8272 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:01<00:00, 18.42it/s]


-------------------------------------------------------------
Build Time: 11.2576 sec, Search Time: 1.3579 sec
Epsilon: 24.5, Total Time: 12.6155 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 18.08it/s]

-------------------------------------------------------------
Build Time: 11.3566 sec, Search Time: 1.3790 sec
Epsilon: 25.0, Total Time: 12.7356 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 11.3433 sec, Search Time: 1.3332 sec, Total Time: 12.6765 sec
Epsilon: 0.5, Build Time: 11.3993 sec, Search Time: 1.2424 sec, Total Time: 12.6417 sec
Epsilon: 1.0, Build Time: 11.4472 sec, Search Time: 1.2711 sec, Total Time: 12.7183 sec
Epsilon: 1.5, Build Time: 11.5366 sec, Search Time: 1.2792 sec, Total Time: 12.8158 sec
Epsilon: 2.0, Build Time: 11.3738 sec, Search Time: 1.3816 sec, Total Time: 12.7554 sec
Epsilon: 2.5, Build Time: 11.3279 sec, Search Time: 1.3702 sec, Total Time: 12.6981 sec
Epsilon: 3.0, Build Time: 11.3100 sec, Search Time: 1.4032 sec, Total Time: 12.7133 sec
Epsilon: 3.5, Build Time: 11.3792 sec, Search Time: 1.3510 sec, Total Time: 12.7301 sec
Epsilon: 4.0, Build Time: 11.3945 sec, Search Time: 1.3687 sec, Total Time: 12.7632 sec
Epsilon: 4.5

**----------Run 2---------------**

In [27]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 8800
epsilon_rebuild = 2.5

leaf_size_insert = 1700

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:01<00:00, 19.26it/s]


-------------------------------------------------------------
Build Time: 11.4364 sec, Search Time: 1.3574 sec
Epsilon: 0.0, Total Time: 12.7938 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 22.03it/s]


-------------------------------------------------------------
Build Time: 11.5347 sec, Search Time: 1.2434 sec
Epsilon: 0.5, Total Time: 12.7781 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.5585 sec, Search Time: 1.4131 sec
Epsilon: 24.0, Total Time: 12.9717 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:01<00:00, 18.27it/s]


-------------------------------------------------------------
Build Time: 11.5853 sec, Search Time: 1.3839 sec
Epsilon: 24.5, Total Time: 12.9692 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:01<00:00, 17.87it/s]

-------------------------------------------------------------
Build Time: 11.4802 sec, Search Time: 1.4118 sec
Epsilon: 25.0, Total Time: 12.8920 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 11.4364 sec, Search Time: 1.3574 sec, Total Time: 12.7938 sec
Epsilon: 0.5, Build Time: 11.5347 sec, Search Time: 1.2434 sec, Total Time: 12.7781 sec
Epsilon: 1.0, Build Time: 11.4826 sec, Search Time: 1.2533 sec, Total Time: 12.7359 sec
Epsilon: 1.5, Build Time: 11.5055 sec, Search Time: 1.3459 sec, Total Time: 12.8514 sec
Epsilon: 2.0, Build Time: 11.3144 sec, Search Time: 1.4160 sec, Total Time: 12.7304 sec
Epsilon: 2.5, Build Time: 11.4450 sec, Search Time: 1.3829 sec, Total Time: 12.8279 sec
Epsilon: 3.0, Build Time: 11.4173 sec, Search Time: 1.4106 sec, Total Time: 12.8278 sec
Epsilon: 3.5, Build Time: 11.5753 sec, Search Time: 1.3703 sec, Total Time: 12.9456 sec
Epsilon: 4.0, Build Time: 11.3577 sec, Search Time: 1.3908 sec, Total Time: 12.7485 sec
Epsilon: 4.5

**--------Epsilon Optimization For Insert---------**

In [28]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [12.6765,12.6417,12.7183,12.8158,12.7554,12.6981,12.7133,12.7301,12.7632,
 12.9318,12.7375,12.7258,12.7119,12.6523,12.749 ,12.7265,12.6364,12.7801,
 12.5159,12.979 ,12.9061,12.7897,12.8006,12.4678,12.7706,12.5476,12.4999,
 12.7328,12.6533,12.7116,12.6645,12.7272,12.9233,12.4562,12.7392,12.725 ,
 12.5714,12.7594,12.8202,12.6228,12.971 ,12.975 ,12.7643,12.6172,12.7506,
 12.6659,12.672 ,12.7449,12.8272,12.6155,12.7356]
)

total_times_2 = np.array(
    [12.7938,12.7781,12.7359,12.8514,12.7304,12.8279,12.8278,12.9456,12.7485,
 13.003 ,12.8479,12.846 ,12.7693,12.7886,13.0648,12.6033,12.7783,12.6438,
 12.6645,12.695 ,12.6482,12.7304,12.9679,12.7167,12.7552,12.9522,12.6444,
 12.8268,12.7816,12.9668,12.6538,12.7995,12.8087,12.926 ,12.9336,12.7519,
 12.8272,12.7982,12.6805,12.6607,12.6197,12.7409,12.8205,12.5923,12.7691,
 12.8772,12.8702,12.8227,12.9717,12.9692,12.892 ]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 12.5722
Minimum found at epsilon size: 13.0
